In [ ]:
!pip install mlflow

In [ ]:
# https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html

In [ ]:
region = 'us-west-2'
app_name = 'sasha-mlflow-test'

In [ ]:
import mlflow.sagemaker

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])
reg

In [ ]:
import mlflow.sklearn
mlflow.sklearn.save_model(reg, "basic_reg")

In [ ]:
#!mlflow sagemaker build-and-push-container --no-push

In [ ]:
#!mlflow sagemaker build-and-push-container --no-build

In [ ]:
# mlflow.sagemaker.push_image_to_ecr()

In [ ]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
role

In [ ]:
# Add S3 object tagging permission

In [ ]:
bucket = sess.default_bucket() 
bucket

In [ ]:
mlflow.sagemaker.deploy("sasha-mlflow-test", "basic_reg", execution_role_arn=role, bucket=bucket, image_url="874389354274.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc:1.11.0", region_name='us-west-2')

In [ ]:
import pandas as pd
data = [[0, 0], [0, 1], [0, 2], [1, 0]]
df = pd.DataFrame(data, columns=['x', 'y'])
df

In [ ]:
df.to_json(orient='split')

In [ ]:
import json
import boto3

def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client("sagemaker-runtime", region)
  
    response = client.invoke_endpoint(
        EndpointName=app_name,
        Body=input_json,
        ContentType='application/json; format=pandas-split',
    )
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    print("Received response: {}".format(preds))
    return preds

In [ ]:
query_endpoint(app_name, df.to_json(orient='split'))

In [ ]:
reg.predict(data)

In [ ]:
def get_active_endpoints(app_name):
    sage_client = boto3.client('sagemaker', region_name=region)
    app_endpoints = sage_client.list_endpoints(NameContains=app_name)["Endpoints"]
    return list(filter(lambda en : en == app_name, [str(endpoint["EndpointName"]) for endpoint in app_endpoints]))
  
print("The following endpoints exist for the `{an}` application: {eps}".format(an=app_name, eps=get_active_endpoints(app_name)))

In [ ]:
mlflow.sagemaker.delete(app_name, region, archive=False)

In [ ]:
print("The following endpoints exist for the `{an}` application: {eps}".format(an=app_name, eps=get_active_endpoints(app_name)))